<a href="https://colab.research.google.com/github/DoubekSeth/CS5350HW/blob/main/DecisionTree/CS5350HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data sets

## Libraries

In [100]:
from collections import Counter
import queue
import math
import statistics
import numpy as np
import pandas as pd

## Datasets

### Tennis

In [2]:
s_tennis = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"]]
full_attributes_tennis = {"outlook":["sunny", "overcast", "rainy"], "temperature":["hot", "mild", "cool"], "humidity":["high", "normal", "low"], "wind":["strong", "weak"]}
remaining_attributes_tennis = ["outlook", "temperature", "humidity", "wind"]

### Boolean Table Example (1st problem)


In [3]:
s_booltab = [
    ["0", "0", "1", "0", "0"],
    ["0", "1", "0", "0", "0"],
    ["0", "0", "1", "1", "1"],
    ["1", "0", "0", "1", "1"],
    ["0", "1", "1", "0", "0"],
    ["1", "1", "0", "0", "0"],
    ["0", "1", "0", "1", "0"]
]
full_attributes_booltab = {"x1":["0", "1"], "x2":["0", "1"], "x3":["0", "1"], "x4":["0", "1"]}
remaining_attributes_booltab = ["x1", "x2", "x3", "x4"]

### Car

In [4]:
s_cartraining = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/car-4/train.csv", header=None).to_numpy()
s_cartesting = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/car-4/test.csv", header=None).to_numpy()

full_attributes_car = {
    "buying":["vhigh", "high", "med", "low"],
    "maint":["vhigh", "high", "med", "low"],
    "doors":["2", "3", "4", "5more"],
    "persons":["2", "4", "more"],
    "lug_boot":["small", "med", "big"],
    "safety":["low", "med", "high"]}
remaining_attributes_car = ["buying", "maint", "doors", "persons", "lug_boot", "safety"]

### Bank

In [92]:
s_banktraining = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-4/train.csv", header=None).to_numpy()
s_banktesting = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/bank-4/test.csv", header=None).to_numpy()

full_attributes_bank = {
    "age":"numeric",
    "job":["admin.","unknown","unemployed","management","housemaid","entrepreneur","student", "blue-collar","self-employed","retired","technician","services"],
    "marital":["married","divorced","single"],
    "education":["unknown","secondary","primary","tertiary"],
    "default":["yes", "no"],
    "balance":"numeric",
    "housing":["yes", "no"],
    "loan":["yes", "no"],
    "contact":["unknown","telephone","cellular"],
    "day":"numeric",
    "month":["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"],
    "duration":"numeric",
    "campaign":"numeric",
    "pdays":"numeric",
    "previous":"numeric",
    "poutcome":["unknown","other","failure","success"]
}
remaining_attributes_bank = ["age", "job", "marital", "education", "default", "balance", "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome"]

# ID3 Algorithm

## Main Algorithm

In [5]:
class Node:
  def __init__(self, s, atrs, parent=None, parentVal=None, label=None):
    self.s = s
    self.atrs = atrs
    self.parent = parent
    self.label = label
    self.children = []
    self.parentVal = parentVal

  def addChild(self, node):
    self.children.append(node)

In [170]:
def ID3(s, remaining_attributes, full_attributes, purity_func, parentNode, level=0, stopping=math.inf):
  #First, check if all training examples s have same label. First stopping condition
  if(labelSame(s)[0]):
    #print("Same Label", labelSame(s)[1])
    parentNode.label = labelSame(s)[1]
    return
  #Second, check if exhausted attributes and if so return leaf node with most common label. Second stopping condition
  if(len(remaining_attributes) == 0):
    #print("Exhausted Attributes", mostCommonLabel(s, 1)[0][0])
    parentNode.label = mostCommonLabel(s, 1)[0][0]
    return
  #Third, check if have hit maximum stopping depth
  #print("hit ", level, " out of ", stopping)
  if(level >= stopping):
    parentNode.label = mostCommonLabel(s, 1)[0][0]
    return
  level += 1
  #If didn't stop, then we can move onto the main part of the algorithm
  #Create root if we don't have one
  if(parentNode==None):
    parentNode = Node(s=s, atrs=remaining_attributes, parent=None)
  #Find attribute best splits S
  bestAtr = findBestSplit(s, remaining_attributes, full_attributes, purity_func)
  parentNode.label = bestAtr
  #print(bestAtr)
  less_remaining_attributes = remaining_attributes[:]
  #print(less_remaining_attributes, bestAtr)
  less_remaining_attributes.remove(bestAtr)
  #Go thru all values for best attribute
  loop_attrs = full_attributes[bestAtr]
  numericFilter=False
  if(loop_attrs=="numeric"):
    numericFilter=True
    loop_attrs = ["lower than median", "higher than median"]
  for val in loop_attrs:
    s_v = filterSOnAtrVal(s, bestAtr, val, full_attributes, numericFilter)
    #Check if s_v empty
    if(len(s_v)==0):
      #print("S_v empty", val, mostCommonLabel(s, 1)[0][0])
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val, label=mostCommonLabel(s, 1)[0][0])
      parentNode.addChild(newNode)
    else:
      newNode = Node(s=s_v, atrs=less_remaining_attributes, parent=parentNode, parentVal=val)
      #print("New node", val)
      parentNode.addChild(newNode)
      ID3(s_v, less_remaining_attributes, full_attributes, purity_func, newNode, level=level, stopping=stopping)
  return parentNode

## Helper Functions

Does two functions, first returns true if all labels are the same for s. Second returns the label if all labels are same or else returns none

In [7]:
def labelSame(s):
  label = s[0][len(s[0])-1]
  for i in s:
    if(i[len(i)-1] != label):
      return False, None
  return True, label

Another helper function, returns the n most common label in s

In [8]:
def mostCommonLabel(s, n):
  labels = []
  for i in s:
    labels.append(i[len(i)-1])
  #Find most common label out of all the labels
  mostCommon = Counter(labels).most_common(n)
  return mostCommon

This helper finds the attribute that results in the best split

In [169]:
def findBestSplit(s, atrs, full_attributes, purity_func):
  max_info_gain = 0
  max_info_gain_atr = atrs[0]
  for atr in atrs:
    info_gained = find_info_gained(s, atr, full_attributes, purity_func)
    #print(atr, info_gained)
    if(info_gained > max_info_gain):
      max_info_gain = info_gained
      max_info_gain_atr = atr
  return max_info_gain_atr

This helper gives the information gained from a single attribute

In [117]:
def find_info_gained(s, atr, full_attributes, purity_func):
  if(full_attributes[atr]=="numeric"):
    return find_numeric_info_gained(s, atr, full_attributes, purity_func)
  else:
    return find_categorical_info_gained(s, atr, full_attributes, purity_func)

In [105]:
def find_numeric_info_gained(s, atr, full_attributes, purity_func):
  atr_ind = list(full_attributes.keys()).index(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {"low":[], "high":[]}
  median = statistics.median(x[atr_ind] for x in s)
  for i in s:
    if i[atr_ind] < median:
      atr_type_count_dict["low"].append(i[len(i)-1])
    else:
      atr_type_count_dict["high"].append(i[len(i)-1])
  starting_info = purity_func(list(x[len(x)-1] for x in s))
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    weighted_sum += len(j)/len(s)*purity_func(j)
  return starting_info-weighted_sum

In [109]:
def find_categorical_info_gained(s, atr, full_attributes, purity_func):
  atr_ind = list(full_attributes.keys()).index(atr)
  #Create dictionary that contains attribute categories and what labels each attribute category has
  atr_type_count_dict = {}
  for i in s:
    if i[atr_ind] in atr_type_count_dict:
      atr_type_count_dict[i[atr_ind]].append(i[len(i)-1])
    else:
      atr_type_count_dict[i[atr_ind]] = [i[len(i)-1]]
  #Use proportions to run info gain function and find total information gain
  starting_info = purity_func(list(x[len(x)-1] for x in s))
  weighted_sum = 0
  for j in atr_type_count_dict.values():
    weighted_sum += len(j)/len(s)*purity_func(j)
  return starting_info-weighted_sum

Filters S based on an attribute value to get S_v

In [118]:
def filterSOnAtrVal(s, atr, val, full_attributes, numericFilter):
  filtered = []
  atr_ind = list(full_attributes.keys()).index(atr)
  for i in s:
    if(numericFilter):
      if(val == "lower than median"):
        if(i[atr_ind] < statistics.median(x[atr_ind] for x in s)):
          filtered.append(i)
      else:
        if(i[atr_ind] >= statistics.median(x[atr_ind] for x in s)):
          filtered.append(i)
    else:
      if(i[atr_ind] == val):
        filtered.append(i)
  return filtered

## Purity Functions

In [12]:
def entropy(labels):
  vec_counts = np.array(list(Counter(labels).values()))
  vec_props = vec_counts/np.sum(vec_counts)
  return -np.dot(vec_props, np.log2(vec_props))

In [13]:
def GiniInd(labels):
  vec_counts = np.array(list(Counter(labels).values()))
  vec_props = vec_counts/np.sum(vec_counts)
  return -np.dot(vec_props, vec_props)

## Display Tree

In [14]:
def unravelTree(q1, q2, childQ, level):
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  while(not q1.empty()):
    node = q1.get()
    childThere = childQ.get()
    firstChild = True
    print("ParentVal: ", node.parentVal)
    print("Label: ", node.label)
    if(not childThere):
      print("----")
    for child in node.children:
      q2.put(child)
      if(firstChild):
        firstChild = False
      else:
        childQ.put(True)
    if(len(node.children)!= 0):
      childQ.put(False)
  level += 1
  q1 = q2
  q2 = queue.Queue()
  if(q1.empty()):
    return
  unravelTree(q1, q2, childQ, level)

# Run ID3

## Set Data

In [173]:
index = 1
s = [s_tennis, s_cartraining, s_banktraining][index]
s_test = ["", s_cartesting, s_banktesting][index]
rem = [remaining_attributes_tennis, remaining_attributes_car, remaining_attributes_bank][index]
full_attr = [full_attributes_tennis, full_attributes_car, full_attributes_bank][index]
pur = [entropy, GiniInd][1]
stop = 5

## Display Tree

In [135]:
root = ID3(s, rem, full_attr, pur, None, stopping=stop)

q = queue.Queue()
q2 = queue.Queue()
q.put(root)
childQ = queue.Queue()
childQ.put(True)


unravelTree(q, q2, childQ, 0)

poutcome
month
job
duration
job
housing
duration
contact
duration
duration
job
marital
duration
job
month
job
job
job
job
job
age
job
job
job
month
job
loan
default
duration
job
job
job
job
job
job
marital
month
job
job
job
job
education
loan
job
job
job
job
job
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  None
Label:  poutcome
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  unknown
Label:  month
ParentVal:  other
Label:  month
ParentVal:  failure
Label:  month
ParentVal:  success
Label:  month
----
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  jan
Label:  job
ParentVal:  feb
Label:  duration
ParentVal:  mar
Label:  job
ParentVal:  apr
Label:  housing
ParentVal:  may
Label:  duration
ParentVal:  jun
Label:  contact
ParentVal:  jul
Label:  duration
ParentVal:  aug
Label:  duration
ParentVal:  sep
Label:  job
ParentVal:  oct
Label:  marital
ParentVal:  nov
Label:  duration
ParentVal:  dec
Label:  job
----
ParentVal:  jan
Label:  no
ParentVal:  feb
Label:  job
ParentVal:  mar
Label:  no
ParentVal:  apr
L

## Evaluate Model

Finds accuracy of model on test data

In [20]:
def findAcc(root, test, full_attr):
  matched = 0
  for i in test:
    matched += travTree(root, i, full_attr)
  return matched/len(test)

Traverses the tree recursively, returns 1 if test label matches model and 0 otherwise

In [146]:
def travTree(root, datapoint, full_attr):
  #Stopping Condition, root no children
  if(len(root.children) == 0):
    if(root.label == datapoint[len(datapoint)-1]):
      #print("matched")
      return 1
    else:
      #print("no match")
      return 0
  #Otherwise, traverse down
  curr_attr = root.label
  do_numeric = False
  if(full_attr[curr_attr] == "numeric"):
    do_numeric = True
  #print("current attribute: ", curr_attr)
  atr_ind = list(full_attr.keys()).index(curr_attr)
  for child in root.children:
    if(do_numeric):
      median = statistics.median(x[atr_ind] for x in root.s)
      #print("numeric: ", median, " ", datapoint[atr_ind])
      if((datapoint[atr_ind] < median and child.parentVal=="lower than median") or (datapoint[atr_ind] >= median and child.parentVal=="higher than median")):
        return travTree(child, datapoint, full_attr)
    else:
      if(child.parentVal == datapoint[atr_ind]):
        #print("Going down on val: ", child.parentVal)
        return travTree(child, datapoint, full_attr)

In [160]:
root = ID3(s, rem, full_attr, pur, None, stopping=stop)

findAcc(root, s_test, full_attr)

0.8652

In [174]:
for i in range(1, 18):
  root = ID3(s, rem, full_attr, pur, None, stopping=i)
  print(i, ": ", findAcc(root, s_test, full_attr))

1 :  0.7032967032967034
2 :  0.7774725274725275
3 :  0.8159340659340659
4 :  0.8667582417582418
5 :  0.9162087912087912
6 :  0.9162087912087912
7 :  0.9162087912087912
8 :  0.9162087912087912
9 :  0.9162087912087912
10 :  0.9162087912087912
11 :  0.9162087912087912
12 :  0.9162087912087912
13 :  0.9162087912087912
14 :  0.9162087912087912
15 :  0.9162087912087912
16 :  0.9162087912087912
17 :  0.9162087912087912
